In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["subcategory"] == "Cost per MWh delivered")
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "Baseload",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quarterly",
    "annually": "Annual",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

# Plot
fig, ax = plt.subplots(figsize=(6, 3))

# First plot pipeline ESC
esc = "pipeline-h2-to-elec"
for label, plot_df in df.query("esc == @esc").groupby("csp"):
    text = "H2 pipeline (with CSP)" if label else "H2 pipeline (no CSP)"
    plot_df.plot(
        x="flexibility",
        y="value",
        ax=ax,
        ls="solid" if label else "dashed",
        marker="o",
        markersize=5,
        label=text,
        fillstyle="full",
        markerfacecolor="white",
        color="green",
    )

# second plot HVDC ESC
esc = "hvdc-to-elec"
for label, plot_df in df.query("esc == @esc").groupby("csp"):
    text = "HVDC (with CSP)" if label else "HVDC (no CSP)"
    plot_df.plot(
        x="flexibility",
        y="value",
        ax=ax,
        ls="solid" if label else "dashed",
        marker="*",
        markersize=7,
        label=text,
        fillstyle="full",
        color="black",
    )

ax.set_axisbelow(True)
ax.set_yticks(np.arange(25, 275, 50), minor=True)
ax.yaxis.grid(which="minor", ls="--")
ax.set_ylabel("Electricity cost [EUR/MWh]")
ax.set_xlabel("Demand matching")
ax.set_ylim(0, 210)  # Magic number 200: Highest value from TN ESC H2 pipeline

handles, labels = ax.get_legend_handles_labels()

ax.legend(
    loc="lower left",
    bbox_to_anchor=(1, 0),
)

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")